#### Instructions:  
1. Libraries allowed: **Python basic libraries, numpy, pandas, scikit-learn (only for data processing), pytorch, and ClearML.**
2. Show all outputs.
3. Submit jupyter notebook and a pdf export of the notebook. Check canvas for detail instructions for the report. 
4. Below are the questions/steps that you need to answer. Add as many cells as needed. 

## Task 2: Finetuning a pretrained NN
Do transfer learning with ResNet18 and compare peforamnce with the hyperparamter-tuned network.

In [1]:
# Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from clearml import Task

# Initialize ClearML Task
task = Task.init(project_name="Transfer Learning", task_name="Fine-tuning ResNet18")

# Define transformations
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert to 3 channels for ResNet
    transforms.Resize((224, 224)),               # ResNet18 expects 224x224 input
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

ClearML Task: overwriting (reusing) task id=36b4accb145e481da51ed984aba0d42a
2024-11-26 12:52:25,415 - clearml.Repository Detection - WARNING - Password protected Jupyter Notebook server was found! Add `sdk.development.jupyter_server_password=<jupyter_password>` to ~/clearml.conf
2024-11-26 12:52:25,455 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/3cacee62609b4c3db95166a6febc8b12/experiments/36b4accb145e481da51ed984aba0d42a/output/log


In [2]:
# Load dataset
dataset = datasets.ImageFolder('train', transform=transform)

# Split into train and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_subset, val_subset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)

In [3]:
# Load pretrained ResNet18
resnet18 = models.resnet18(pretrained=True)

# Modify the final fully connected layer for the specific task
num_classes = 7  # Replace with the number of classes in your dataset
resnet18.fc = nn.Linear(resnet18.fc.in_features, num_classes)

# Transfer model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = resnet18.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet18.parameters(), lr=0.01, momentum=0.9, weight_decay=0.001)


C:\Users\smitp\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\smitp\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


2024-11-26 12:52:30,626 - clearml.model - INFO - Selected model id: 16743848619946ebb16f4a7b3cd6b877


In [4]:
from torchvision.models import resnet18

# Initialize ResNet18
resnet18 = resnet18(pretrained=True)  # Use pretrained weights
resnet18.fc = nn.Linear(resnet18.fc.in_features, 7)  # Modify the final layer for 7 classes
resnet18 = resnet18.to(device)  # Move to GPU or CPU
# Define criterion and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet18.parameters(), lr=0.1, momentum=0.8)



C:\Users\smitp\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\smitp\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


2024-11-26 12:52:32,027 - clearml.model - WARNING - Connecting multiple input models with the same name: `resnet18-f37072fd`. This might result in the wrong model being used when executing remotely


In [5]:
# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0

        # Training loop
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Validation loop
        model.eval()  # Set the model to evaluation mode
        val_loss = 0.0
        y_true, y_pred = [], []

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                y_true.extend(labels.cpu().numpy())
                y_pred.extend(predicted.cpu().numpy())

        # Metrics
        accuracy = accuracy_score(y_true, y_pred)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, "
              f"Val Loss: {val_loss/len(val_loader):.4f}, Val Accuracy: {accuracy:.4f}")

    return model

In [6]:
# Train and validate ResNet18
trained_resnet18 = train_model(resnet18, train_loader, val_loader, criterion, optimizer, num_epochs=5)


ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start
Epoch 1/5, Loss: 2.2077, Val Loss: 1.9026, Val Accuracy: 0.2464
Epoch 2/5, Loss: 1.8047, Val Loss: 1.7878, Val Accuracy: 0.2489
Epoch 3/5, Loss: 1.7872, Val Loss: 1.7714, Val Accuracy: 0.2619
Epoch 4/5, Loss: 1.7729, Val Loss: 1.7595, Val Accuracy: 0.2706
Epoch 5/5, Loss: 1.7257, Val Loss: 1.6554, Val Accuracy: 0.3384


In [8]:
# Load the saved trained model
from torchvision.models import resnet18

trained_resnet18 = resnet18(weights="IMAGENET1K_V1")  # Use pretrained weights
trained_resnet18.fc = nn.Linear(trained_resnet18.fc.in_features, 7)  # Modify for 7 classes
trained_resnet18 = trained_resnet18.to(device)


# Load test dataset
test_dataset = datasets.ImageFolder('test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [9]:
# Evaluation function
def evaluate_model(model, test_loader):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    # Compute metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

In [10]:
# Evaluate ResNet18
evaluate_model(trained_resnet18, test_loader)

Accuracy: 0.0375
Precision: 0.0932
Recall: 0.1404
F1 Score: 0.0360


C:\Users\smitp\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Discussion
Provide a comparative analysis.

### 1. Model Summary
#### Custom CNN:

A shallow CNN with 2 convolutional layers and 128 fully connected units.

Designed and trained from scratch specifically for the emotion detection task.
#### Fine-tuned ResNet18:

Pretrained ResNet18 modified for 7-class classification.

Transfer learning applied, leveraging ImageNet-pretrained weights.
### 2. Training Metrics
#### Custom CNN:

Final training loss: 1.2913.
Final validation loss: 1.3199.
Validation accuracy: 49.44%.
#### Fine-tuned ResNet18:

Final training loss: 1.7257.
Final validation loss: 1.6554.
Validation accuracy: 33.84%.
#### Observation:

The Custom CNN outperformed ResNet18 in both training and validation, achieving better accuracy and lower losses.
### 3. Test Performance
#### Custom CNN:

Accuracy: 49.26%.
Precision: 48.71%.
Recall: 41.50%.
F1 Score: 40.56%.
#### Fine-tuned ResNet18:

Accuracy: 3.75%.
Precision: 9.32%.
Recall: 14.04%.
F1 Score: 3.60%.
#### Observation:

Custom CNN achieved better test performance across all metrics.

ResNet18 struggled, showing extremely low accuracy and undefined precision for some classes.
### 4. Key Observations
#### Custom CNN Strengths:

Consistently performed better across validation and test datasets.

Tailored architecture better suited for the dataset and task.
#### Fine-tuned ResNet18 Issues:

Mismatch between pretrained weights (ImageNet) and the dataset may have contributed to poor performance.

Insufficient fine-tuning due to limited epochs and potential overfitting to the training data.

### Conclusion
Custom CNN showed significantly better performance, making it more suitable for this task.

Fine-tuned ResNet18 underperformed, but with additional tuning and adjustments, it has potential for improvement due to its capacity and pretrained knowledge.